
Reccap
---

# 📝 **Docker Build & Cache Issues - Recap & Solutions**  

## **1️⃣ Problem: Changes Not Reflecting in Container**  
### **Issue**  
- You made changes to `app.py`, but when you ran your Docker container, the updates were not visible.  
- This happened because **Docker cached previous layers** and did not copy the latest `app.py` into the image.  

### **Solution**  
To force Docker to rebuild the image and apply the latest changes:  
```bash
docker build --no-cache -t kenys-gdp-app .
```
- The `--no-cache` flag ensures that all layers are **rebuilt from scratch**, preventing outdated files from being used.  
- Then, run your container again:  
```bash
docker run -d -p 8601:8501 --name kenys-gdp-container kenys-gdp-app
```
---
  
## **2️⃣ Problem: Container Already Running or Exists**  
### **Issue**  
- When you tried to start a new container, Docker complained that the container name (`kenys-gdp-container`) was already in use.  
- This happens when a container with the same name is **already running or exists in a stopped state**.  

### **Solution**  
**Step 1: Check running containers**  
```bash
docker ps
```
- If the container is running, stop it:  
```bash
docker stop kenys-gdp-container
```
- Remove the container:  
```bash
docker rm kenys-gdp-container
```
- Now, start a new container:  
```bash
docker run -d -p 8601:8501 --name kenys-gdp-container kenys-gdp-app
```

**Alternatively**, if you just want to restart the existing container:  
```bash
docker restart kenys-gdp-container
```
---
  
## **3️⃣ Problem: Port Already in Use**  
### **Issue**  
- When starting the container, you got the error:  
  ```
  Bind for 0.0.0.0:8601 failed: port is already allocated
  ```
- This means another process (possibly an old container) is already using port **8601**.  

### **Solution**  
Find which container is using the port:  
```bash
docker ps
```
Stop the container that’s using the port:  
```bash
docker stop <container_id>
```
If you need to **free the port manually**, run:  
```bash
sudo lsof -i :8601
sudo kill -9 <PID>
```
Now, run your container again.  
---
  
## **4️⃣ Problem: File Not Found Error in Docker**  
### **Issue**  
- Your app inside Docker failed to load `Kenya_GDP_Complete.csv` because it was using a **Windows path (`C:\Users\...`)**.  
- Inside Docker (which is a Linux environment), absolute paths **must use Unix-style paths (`/app/data/...`)**.  

### **Solution**  
Modify your data loading function to use a relative path:  
```python
import os

@st.cache_data
def load_data():
    try:
        # Use relative path
        data_path = os.path.join("data", "Kenya_GDP_Complete.csv")
        data = pd.read_csv(data_path)
        return data
    except Exception as e:
        st.error(f"Error loading data: {e}")
        return None
```
---
  
## **5️⃣ Best Practices to Avoid These Issues**
✅ **Always check running containers before creating a new one**  
```bash
docker ps
```
✅ **Stop and remove old containers before rebuilding**  
```bash
docker stop kenys-gdp-container && docker rm kenys-gdp-container
```
✅ **Use `--no-cache` when rebuilding if changes are not reflecting**  
```bash
docker build --no-cache -t kenys-gdp-app .
```
✅ **Use relative paths instead of absolute Windows paths inside Docker**  
✅ **Ensure ports are free before running new containers**  

---
### 🔥 **Now, you have a clear guide to debugging Docker build issues! 🚀**  
Let me know if you need further clarification. 😃



<center><B>RECCAP</center></b>







Refined step-by-step breakdown:  

---
---


### ✅ **Steps to Ensure Docker Uses the Latest Changes**  

1️⃣ **Check if there is any running container**  
```bash
docker ps
```

2️⃣ **Stop the running container (if any exists)**  
```bash
docker stop kenys-gdp-container
```

3️⃣ **Remove the stopped container**  
```bash
docker rm kenys-gdp-container
```

4️⃣ **Rebuild the Docker image**  
- If cache is enabled and changes are not reflecting, force a rebuild:  
```bash
docker build --no-cache -t kenys-gdp-app .
```
- Otherwise, a normal rebuild should work:  
```bash
docker build -t kenys-gdp-app .
```

5️⃣ **Run the container again**  
```bash
docker run -d -p 8601:8501 --name kenys-gdp-container kenys-gdp-app
```

---

### **💡 Additional Notes:**
🔹 If you get a **port conflict**, first check which process is using it:  
```bash
sudo lsof -i :8601
```
Then kill the process if necessary:  
```bash
sudo kill -9 <PID>
```

🔹 If you just want to restart instead of rebuilding, use:  
```bash
docker restart kenys-gdp-container
```



\

### **Best Practice Instead of Restarting Bash**
👉 **Use `docker system prune` to clean up old cache and rebuild cleanly**
```bash
docker system prune -a -f
```
This removes:
- Unused images
- Stopped containers
- Old build cache

👉 **Force rebuild the image without cache**
```bash
docker build --no-cache -t kenys-gdp-app .
```

---



Yes, every time you **rebuild a Docker image**, it consumes more **disk space** because:  

1. **New image layers are created** (even if they are similar to the previous ones).  
2. **Old unused images remain** unless explicitly removed.  
3. **Container logs and cache** can accumulate over time.  

### **How to Fix & Manage Storage Efficiently**
#### **1. Regularly Remove Unused Resources**
- Use this command to **remove dangling images** (images not linked to any container):  
  ```bash
  docker image prune -f
  ```
- Remove **all unused containers, images, and networks**:  
  ```bash
  docker system prune -f
  ```
- If you want to remove **everything**, including **stopped containers and all unused images**:  
  ```bash
  docker system prune -a -f
  ```
  **⚠ Be careful!** This removes everything **not in use**.

---

#### **2. Use Docker’s Build Cache Efficiently**
- **Avoid unnecessary rebuilds**: If you change only one file, don’t rebuild everything.  
- **Leverage multi-stage builds** in your `Dockerfile` to keep only necessary files.  
- **Use a `.dockerignore` file** to exclude unnecessary files (e.g., logs, `.git`, `node_modules`, etc.).  

---

#### **3. Reuse Image Layers Instead of Full Rebuilds**
Instead of running:  
```bash
docker build -t myapp .
```
Use:  
```bash
docker build --no-cache -t myapp .
```
only when necessary to force a fresh rebuild. Otherwise, let Docker cache layers.

---

#### **4. Remove Old Images Manually**
Check all images:  
```bash
docker images
```
Then remove old ones:  
```bash
docker rmi <image_id>
```

---

### **Conclusion**
- Yes, rebuilding without cleanup consumes storage.  
- Use `docker system prune` commands wisely.  
- Optimize your `Dockerfile` and cache usage.  

Would you like help optimizing your `Dockerfile` for better caching? 🚀